# Inhibitor Diffusion and Germination - Multiple Spores

## Numerical experiments - Week 50/2024

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

## Prerequisite libraries

From this point on, the code is structured in Python modules, which are imported into the notebooks.

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, cuda, float32
import matplotlib.gridspec as gridspec

from importlib import reload
import setup as st
import diffusion as df
import conversions as conv

In [50]:
reload(st)
reload(df)
reload(conv)

<module 'conversions' from 'c:\\Users\\mihbo\\Documents\\GitHub\\msc-thesis\\Code\\conversions.py'>

In [51]:
N = 127
dx = 5
spore_density = 1e9 # spores per mL
converted_density = conv.inverse_mL_to_micrometers_cubed(spore_density)

spore_coords = st.populate_spore_grid_coords(N, dx, spore_density)

# print(spore_coords[0].shape)

Conversion: 0.00100000000000000 micrometers^-3
635
Populating volume of 256047875 micrometers^3 with 2.56047875e+17 spores, 635000.0 spores per dimension
Spore spacing: 0.001 micrometers
